In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor



from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [37]:
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')

In [38]:
df_train['date'] = df_train['date'].apply(lambda x: x[5:7]).astype(int)
df_test['date'] = df_test['date'].apply(lambda x: x[5:7]).astype(int)

In [39]:
train_all_x = df_train.iloc[:, :24]
train_all_y = df_train.iloc[:, 24:]
train_all_x_d = train_all_x

In [40]:
np.isnan(train_all_x['build_tech']).astype(int).sum()

30109

In [41]:
def no_more_nan(train_all_x):
    train_all_x['build_tech'] = train_all_x['build_tech'].fillna(train_all_x['build_tech'].mean())
    train_all_x['n_photos'] = train_all_x['n_photos'].fillna(train_all_x['n_photos'].mean())
    train_all_x['metro_dist'] = train_all_x['metro_dist'].fillna(train_all_x['metro_dist'].mean())
    train_all_x = train_all_x.fillna(0)
    
    return train_all_x

In [42]:
train_all_x = no_more_nan(train_all_x)
df_test = no_more_nan(df_test)

In [44]:
train_all_x.head()

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13
0,0,1,531,0.540942,12,35,1,1,25.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,385,2.000000,15,46,1,2,15.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,51,0.000000,1,53,3,0,30.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,76,0.000000,1,60,3,0,30.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,114,0.000000,2,43,2,0,30.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
reg_ad = LinearRegression().fit(train_all_x, train_all_y)
reg_to_ad = reg_ad.predict(train_all_x)
reg_ad_test = reg_ad.predict(df_test)
len(reg_to_ad)

100000

In [46]:
train_all_x['reg'] = reg_to_ad
df_test['reg'] = reg_ad_test

In [47]:
def kw_all(df):
    df['kw_all'] = df['kw1']+df['kw2']+df['kw3']+df['kw4']+df['kw5']+df['kw6']+df['kw7']+df['kw8']+df['kw9']+df['kw10']+df['kw11']+df['kw12']+df['kw13']
    return df

In [50]:
train_all_x = kw_all(train_all_x)
df_test = kw_all(df_test)

In [48]:
xgb_ad_8_02_400_23 = XGBRegressor(max_depth=8, learning_rate=0.2, n_estimators=50,min_child_weight =23, eval_metric= 'mae').fit(train_all_x, train_all_y)
xgb_pred_8_02_400_23 = xgb_ad_8_02_400_23.predict(train_all_x)
xgb_pred = xgb_ad_8_02_400_23.predict(df_test)

[12:57:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [49]:
train_all_x['xgb_ad_8_02_400_23'] = xgb_pred_8_02_400_23
df_test['xgb_ad_8_02_400_23'] = xgb_pred

In [51]:
X_train,X_test,y_train,y_test = train_test_split(train_all_x, train_all_y.astype(int), test_size=0.25, random_state=42, shuffle = False)

In [37]:
#train_all_x.corr()

In [52]:
reg = LinearRegression().fit(X_train, y_train)
reg_pred = reg.predict(X_test)
mean_absolute_error(reg_pred, y_test)

1105114.1738509112

In [76]:
%%time
xgb = XGBRegressor(max_depth=8, learning_rate=0.2, n_estimators=400, min_child_weight =23, eval_metric= 'mae', random_state = 1).fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_mae = mean_absolute_error(xgb_pred, y_test)
print(xgb_mae)

[17:31:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
906024.83277
Wall time: 1min 56s


In [ ]:
kw_all + reg(train_all+xgb) +xgb(train_all)

906024.83277 --max_depth=8, learning_rate=0.2, n_estimators=400, min_child_weight =23

921573.664865 --max_depth=8, learning_rate=0.2, n_estimators=300, min_child_weight =25
950581.7382125 --max_depth=8, learning_rate=0.2, n_estimators=300, min_child_weight =24
920085.0162775 --max_depth=8, learning_rate=0.2, n_estimators=300, min_child_weight =22
940261.5057675 --max_depth=8, learning_rate=0.2, n_estimators=300, min_child_weight =21
940682.38684 --max_depth=8, learning_rate=0.2, n_estimators=300, min_child_weight =20

941096.50214 --max_depth=8, learning_rate=0.22, n_estimators=300,min_child_weight =23
918824.166525 --max_depth=8, learning_rate=0.18, n_estimators=300,min_child_weight =23
918190.210065 --max_depth=8, learning_rate=0.19, n_estimators=300,min_child_weight =23
937727.386685 --max_depth=8, learning_rate=0.21, n_estimators=300,min_child_weight =23

+905850.62393 --max_depth=8, learning_rate=0.2, n_estimators=300,min_child_weight =23 
932375.1894525 --max_depth=10, learning_rate=0.2, n_estimators=300,min_child_weight =23
942567.06822 --max_depth=6, learning_rate=0.2, n_estimators=300,min_child_weight =23
910863.4241775 --max_depth=7, learning_rate=0.2, n_estimators=300,min_child_weight =23
930725.5708075 --max_depth=9, learning_rate=0.2, n_estimators=300,min_child_weight =23
    

In [ ]:
kw_all + reg(train_all)

962850.4630814062 --max_depth=8, learning_rate=0.2, n_estimators=400,min_child_weight =23

985019.9811825 --max_depth=8, learning_rate=0.2, n_estimators=300,min_child_weight =20
*967587.150561875 --max_depth=8, learning_rate=0.2, n_estimators=300,min_child_weight =23
984436.206763125 --max_depth=8, learning_rate=0.2, n_estimators=300,min_child_weight =24
__
1361579.9856875 --max_depth=8, learning_rate=0.22, n_estimators=50,min_child_weight =23
1391768.97323 --max_depth=8, learning_rate=0.2, n_estimators=50,min_child_weight =24
1380256.7940575 --max_depth=8, learning_rate=0.2, n_estimators=50,min_child_weight =25
1360917.22109 --max_depth=8, learning_rate=0.22, n_estimators=50,min_child_weight =22
1350778.610455 --max_depth=8, learning_rate=0.22, n_estimators=50,min_child_weight =21
-1325919.698035 --max_depth=8, learning_rate=0.22, n_estimators=50,min_child_weight =20
1331409.3910325 --max_depth=8, learning_rate=0.22, n_estimators=50,min_child_weight =19
1346910.936505 --max_depth=8, learning_rate=0.22, n_estimators=50,min_child_weight =18

1379874.3758425 --max_depth=8, learning_rate=0.2, n_estimators=50,min_child_weight =231379874.3758425
1391599.5824975 --max_depth=8, learning_rate=0.19, n_estimators=50,min_child_weight =23
-1361579.9856875 --max_depth=8, learning_rate=0.22, n_estimators=50,min_child_weight =23
__
992391.1315125 --max_depth=8, learning_rate=0.21, n_estimators=300,min_child_weight =23
976469.985785 --max_depth=8, learning_rate=0.19, n_estimators=300,min_child_weight =23

1023949.1333875 --max_depth=12, learning_rate=0.2, n_estimators=300,min_child_weight =23
1001002.139025 --max_depth=11, learning_rate=0.2, n_estimators=300,min_child_weight =23
1000598.8239375 --max_depth=10, learning_rate=0.2, n_estimators=300,min_child_weight =23
966723.50422375 --max_depth=9, learning_rate=0.2, n_estimators=300,min_child_weight =23
+967587.150561875 --max_depth=8, learning_rate=0.2, n_estimators=300,min_child_weight =23
972168.9276775 --max_depth=7, learning_rate=0.2, n_estimators=300,min_child_weight =23

In [55]:
kw_all = all 

++1040354.3905878125 --max_depth=12, learning_rate=0.2, n_estimators=300,min_child_weight =23

1059796.392741875 --max_depth=12, learning_rate=0.2, n_estimators=300,min_child_weight =24
1072012.5983840625 --max_depth=12, learning_rate=0.2, n_estimators=300,min_child_weight =25
1065981.525840078 --max_depth=12, learning_rate=0.2, n_estimators=300,min_child_weight =22
    
1046936.04384 --max_depth=12, learning_rate=0.2, n_estimators=200,min_child_weight =23
1046141.178086875 --max_depth=12, learning_rate=0.2, n_estimators=250,min_child_weight =23
1041415.3766671875 --max_depth=12, learning_rate=0.2, n_estimators=350,min_child_weight =23

+1040354.3905878125 --max_depth=12, learning_rate=0.2, n_estimators=300,min_child_weight =23, eval_metric= 'mae'
1054798.586728125 --max_depth=12, learning_rate=0.18, n_estimators=300,min_child_weight =23
1078943.7528842578 --max_depth=12, learning_rate=0.19, n_estimators=300,min_child_weight =23
1082112.002559922 --max_depth=12, learning_rate=0.21, n_estimators=300,min_child_weight =23
1121473.7090925 --max_depth=12, learning_rate=0.25, n_estimators=300,min_child_weight =23

1104826.513430547 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =21
1074625.5027826563 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =22
1076312.278398125  --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =23
+1066766.8459231288 --max_depth=12, learning_rate=0.23, n_estimators=300,min_child_weight =23
1088418.7727475 --max_depth=13, learning_rate=0.23, n_estimators=300,min_child_weight =23

SyntaxError: invalid syntax (<ipython-input-55-ce53ba3af53c>, line 2)

In [ ]:
kw_all == 10+13
1052256.1394975 -max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =21,gamma = 30
1052256.1394975 -max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =21,gamma = 50
1052256.1394975  -max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =21,gamma = 500

no change
1129773.1731574219 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =10
1125427.0251547266 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =15
1067645.7519520458 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =20
1097996.3700570704 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =25
+*1048835.781024375 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =22
1105452.059796875 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =23
1085935.9652590626 --max_depth=11, learning_rate=0.23, n_estimators=300,min_child_weight =21

1194588.359026875 -- max_depth=11, learning_rate=0.23, n_estimators=200
1189501.2317166405 --max_depth=11, learning_rate=0.23, n_estimators=300

n_estimators=100
1359123.300235 -- max_d = 11, rate =0.1
1341221.0255725 -- max_d = 11, rate =0.11
1283818.43950625  -- max_d = 11, rate =0.15
1271262.40752375 -- max_d = 11, rate =0.2
1245637.9609341796 -- max_d = 11, rate =0.25
1277047.9388689843 ---- max_d = 11, rate =0.3
1227098.794810625---- max_d = 11, rate =0.23
1225849.891115

rate =0.1 n_estimators=100
1588391.82617 -- max_d =5
1408624.671765 -- max_d = 7
1362923.933185 -- max_d = 9
1359123.300235 -- max_d = 11
1545709.944415  -- mad_d =15

In [133]:
df_test.dtypes

id              int64
date           object
street_id       int64
build_tech    float64
floor           int64
area            int64
rooms           int64
balcon          int64
metro_dist    float64
g_lift        float64
n_photos        int64
kw1             int64
kw2             int64
kw3             int64
kw4             int64
kw5             int64
kw6             int64
kw7             int64
kw8             int64
kw9             int64
kw10            int64
kw11            int64
kw12            int64
kw13            int64
dtype: object

In [ ]:
max_depth=8, learning_rate=0.2, n_estimators=300,min_child_weight =23 

In [72]:
xgb = XGBRegressor(max_depth=8, learning_rate=0.2, n_estimators=300,min_child_weight =23, eval_metric= 'mae').fit(train_all_x, train_all_y)
xgb_pred = xgb.predict(df_test)
#xgb_mae = mean_absolute_error(xgb_pred, y_test)
#xgb_mae

[13:31:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [73]:
df_test["price"] = xgb_pred

df_test[["id", "price"]].to_csv("xgb_8_2_300_23_all_reg_xgb.csv", index=False)